<a href="https://colab.research.google.com/github/rawatnikhil857/knightRiders-hackon/blob/main/LoRA_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LoRA Implementation

### Install Dependencies
Libraries used - accelerate, loralib, PEFT from hugging face.
PEFT gives access to LoRA(low rank adaptation) which is a way to efficiently fine tune Large Language models

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.0 MB/s eta 0

#### Confirm CUDA

In [ ]:
import torch
torch.cuda.is_available()

True

#### Load Base Model

Bloom 1b7 is used as a base LLM to fine tune upon. bloom-1b7 is a Transformer-based Language Model. And we're using bigscience/tokenizer which is a tokenizer used for all BLOOM models

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    torch_dtype=torch.float16,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")

##### View Model Summary
we require model summary in order to choose which module will be used as a target for fine tuning which varies according to the base model used. In bloom-1b7 we'll be targetting "query_key_value" module.

In [ ]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (

In [ ]:
#pre-processing step to help improve training stability
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

#### Helper Function

In [ ]:
#visualize LoRA
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Obtain LoRA Model


In [ ]:
from peft import LoraConfig, get_peft_model
#LoraConfig used to set rank, module targeted for fine tuning
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config) #we get our PEFT model with LoRA configurations
print_trainable_parameters(model)

trainable params: 1572864 || all params: 1723981824 || trainable%: 0.09123437254985815


We can see above how we're only required to train 0.09% of all parameters in our large language model using LoRA

#### Load Sample Dataset


In [ ]:
from datasets import load_dataset
qa_dataset = load_dataset('csv', data_files='Context-Question-AnswerDatabase.csv')
type(qa_dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

datasets.dataset_dict.DatasetDict

Below is the structure of prompt used to fine tune our model. We make the language model to learn the structure of the query that will be made and the structure of the answer it's supposed to generate by providing it multiple such prompts and training the parameters accordingly.

```
### CONTEXT
{context}

### QUESTION
{question}

### ANSWER
{answer}</s>
```

In [ ]:
# A function to map data to a specific prompt
def create_prompt(context, question, answer):
  prompt_template = f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n{answer}</s>"
  return prompt_template

mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['Context'], samples['Question'], samples['Answer'])))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
mapped_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Context', 'Question', 'Answer', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

#### Train LoRA

In [ ]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_qa_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.471600
2,1.663600
3,1.431500
4,1.477500
5,1.384000
6,1.635900
7,1.337200
8,1.470500
9,1.518300
10,1.395000


TrainOutput(global_step=100, training_loss=1.060699276328087, metrics={'train_runtime': 493.102, 'train_samples_per_second': 0.811, 'train_steps_per_second': 0.203, 'total_flos': 2788108539985920.0, 'train_loss': 1.060699276328087, 'epoch': 0.4})

We can save above trained model to hugging face by logging in as shown below

In [ ]:
HUGGING_FACE_USER_NAME = "rawatnikhil857"
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "movieRec-bloom-1b7"
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:845: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/6.31M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rawatnikhil857/movieRec-bloom-1b7-v2/commit/9ca1dfdd6f6df6e79322eefc5f25d075570758cc', commit_message='Upload model', commit_description='', oid='9ca1dfdd6f6df6e79322eefc5f25d075570758cc', pr_url=None, pr_revision=None, pr_num=None)

We can see how our parameter efficient fine tuned model is only 6.31 Mb and we can access it again from hugging face without needing to retrain entire base model
